# First look at the SkySim5000 mass function
Authors : Constantin Payerne, Céline Combet

In [ ]:
import numpy as np
import pyccl as ccl
import matplotlib.pyplot as plt
import pickle as pkl
import scipy.integrate
import astropy.units as u
import GCRCatalogs

%matplotlib inline

In [ ]:
skysim_cat = GCRCatalogs.load_catalog('skysim5000_v1.1.1_small')

In [ ]:
cosmo_ss  = skysim_cat.cosmology

In [ ]:
cosmo_ss

## Extract DM haloes from the catalog in a given mass and redshift range. 

In [ ]:
# get list of halos in a given redshift and mass range 
mmin_extract = 1.e12 # Msun (M_fof)
zmin_extract = 0.
zmax_extract = 1.0

dm_halos = skysim_cat.get_quantities(['halo_mass','hostHaloMass','redshift','ra', 'dec', 'halo_id',
                                             'baseDC2/sod_halo_mass','baseDC2/sod_halo_radius'],
                                            filters=[f'halo_mass > {mmin_extract}','is_central==True',
                                            f'redshift>{zmin_extract}', f'redshift<{zmax_extract}'])

In [ ]:
N_cl = len(dm_halos['halo_mass'])
print(f'There are {N_cl} halos in this mass (Mfof) and redshift range')

In [ ]:
plt.scatter(dm_halos['ra'], dm_halos['dec'], marker='.', s=0.001)
plt.xlabel('ra [deg]')
plt.ylabel('dec [deg]')

## Define a redshift and mass range for the comparison of data and prediction, and filter the data accordingly
NB: SkySim5000 M200c masses are in units of Msun/h

In [ ]:
tmp = dm_halos['baseDC2/sod_halo_mass']/cosmo_ss.h # sod_halo_mass = M200,c in Msun/h, needs conversion
zmin = 0.
zmax = 1.
mmin = np.min(tmp[tmp>0]) # Msun. Filtering are there are some negative masses in the table
mmax = 1.e15 # Msun
filt1 = dm_halos['redshift'] >= zmin
filt2 = dm_halos['redshift'] <= zmax
 
filt3 = tmp >= mmin
filt4 = tmp < mmax
filt = filt1 * filt2 * filt3 * filt4

data = dm_halos['baseDC2/sod_halo_mass'][filt]/cosmo_ss.h # M200,c [Msun]

In [ ]:
nbins = 15
plt.figure(figsize=(10,6))
hist = plt.hist(np.log10(data), bins=nbins)
plt.yscale('log')
plt.xlabel('M200,c [Msun]', size=12)
plt.ylabel('Number of halos', size=12)
print(f"Total number of halos in z=[{zmin},{zmax}] and M=[{mmin/1.e14}, {mmax/1.e15}] x 1e15 Msun = {len(data)}")
hist

## Prediction using CCL and the Tinker08 and Bocquet16 mass functions

In [ ]:
# Define CCL Cosmology from SkySim cosmology
cosmo = ccl.Cosmology(Omega_c=cosmo_ss.Om0-cosmo_ss.Ob0, Omega_b=cosmo_ss.Ob0,
                      h=cosmo_ss.h, sigma8=cosmo_ss.sigma8, n_s=cosmo_ss.n_s, Neff=3.04)

print(cosmo)


### Differential comoving volume

In [ ]:
def dV_over_dOmega_dz(z):
    a = 1./(1. + z)
    da = ccl.background.angular_diameter_distance(cosmo, a) 
    E = ccl.background.h_over_h0(cosmo, a)
    return ((1.+z)**2)*(da**2)*ccl.physical_constants.CLIGHT_HMPC/cosmo['h']/E 

In [ ]:
### Sanity check - comparison to the volume computed with Astropy
dV_over_dOmega_dz(0.3), cosmo_ss.differential_comoving_volume(0.3).value 

### CCL mass functions

In [ ]:
hmd_200c = ccl.halos.MassDef(200, 'critical')
def tinker08(logm, z):
    mass = 10**(logm)
    hmf_200c = ccl.halos.MassFuncTinker08(cosmo, mass_def=hmd_200c)
    nm = hmf_200c.get_mass_function(cosmo, mass, 1./(1+z))
    return nm # dn/dlog10M

hmd_200c = ccl.halos.MassDef200c()

def bocquet16(logm, z):
    mass = 10**(logm)
    hmf_200c = ccl.halos.MassFuncBocquet16(cosmo, mass_def=hmd_200c)
    nm = hmf_200c.get_mass_function(cosmo, mass, 1./(1+z))
    return nm # dn/dlog10M

In [ ]:
def integrand_tinker08(logm,z):
    return tinker08(logm, z)*dV_over_dOmega_dz(z)

def integrand_bocquet16(logm,z):
    return bocquet16(logm, z)*dV_over_dOmega_dz(z)

### Solid angle of the `small` catalog = 50 deg2

Need to check if it is exactly 50 deg2 or if this has been rounded

In [ ]:
DeltaOmega = 50 * np.pi**2/180**2

In [ ]:
N_predicted_T08 = []

# Loop over mass bins defined by plt.hist and predict the corresponding number of haloes
# for Tinker 08
for i in np.arange(len(hist[0])):
    logmmin = np.log10(10**hist[1][i]) 
    logmmax = np.log10(10**hist[1][i+1])
    N_T08 = scipy.integrate.dblquad(integrand_tinker08, zmin, zmax, lambda x:logmmin, lambda x:logmmax, epsabs=1.e-4, epsrel=1.e-4)
    N_predicted_T08.append(N_T08[0]) 

In [ ]:
N_predicted_B16 = []

# Loop over mass bins defined by plt.hist and predict the corresponding number of haloes
# for Bocquet 16
for i in np.arange(len(hist[0])):
    logmmin = np.log10(10**hist[1][i]) # Msun
    logmmax = np.log10(10**hist[1][i+1]) # Msun
    N_B16 = scipy.integrate.dblquad(integrand_bocquet16, zmin, zmax, lambda x:logmmin, lambda x:logmmax, epsabs=1.e-5, epsrel=1.e-4)
    N_predicted_B16.append(N_B16[0]) 

## Plot measured versus predicted number of haloes

In [ ]:
plt.figure(figsize=(10,6))
hist = plt.hist(np.log10(data), bins=nbins, 
                label=f'SkySim5000, 50 deg2 field, z=[{zmin},{zmax}]', histtype='bar', alpha=0.5)
bin_centers=[(hist[1][i]+hist[1][i+1])/2. for i in np.arange(len(hist[0]))]
plt.plot(bin_centers,np.array(N_predicted_T08)*DeltaOmega, 'x-', label='T08, CCL')
plt.plot(bin_centers,np.array(N_predicted_B16)*DeltaOmega, '.-', label='B16, CCL')
plt.legend(fontsize=12)
plt.yscale('log')
plt.xlabel('$M_{200c}$ [M$_\odot$]', size=14)
plt.ylabel('Number of haloes', size=14)